In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
with open('urls.json') as fopen:
    urls = sorted(json.load(fopen))
    
len(urls)

5637

In [4]:
r = requests.get(urls[0])
soup = BeautifulSoup(r.content, "lxml")

In [9]:
soup.find('div', {'class': 'stitle'}).text

'7-Eleven Malaysia Launches New My7E Loyalty App'

In [11]:
soup.find('div', {'class': 'scontent portfolio__content'}).get_text(separator="\n")

'\n\n\n\n7-Eleven Malaysia has launched its own loyalty app called My7E. According to the 24/7 convenience store chain, users get to enjoy special rewards and exclusive discounts via this new membership app.\n\n\nMy7E allows you to collect points by presenting your in-app barcode to the cashier when you make a purchase at 7-Eleven. The app also rewards members who make use of the app’s e-stamp cards, where repeat purchases from selected brands earn you various benefits. The My7E app has e-coupon deals with discounted offers that you can take advantage of as well.\n\n\n\n\n\n\n\n\n\n\nFurthermore, you can link your B Infinite card and Razer Pay e-wallet to the My7E app to earn points from those rewards programmes too. The Razer Pay e-wallet can also be used to make cashless payments via the app.\n\n\nCurrently, the My7E app is offering RM3 e-cash vouchers to new users who sign up for the application. Besides that, users will also get one free Slurpee for every six Slurpee drinks purchas

In [12]:
!mkdir page

In [13]:
for no, url in tqdm(enumerate(urls)):
    filename = os.path.join('page', f'{no}.json')
    if os.path.exists(filename):
        continue
    
    success = False
    while True:
        try:
            r = requests.get(url)
            success = True
            break
        except Exception as e:
            if 'xceeded 30 redirects' in str(e):
                break
            time.sleep(1.0)
            
    if not success:
        continue
            
    soup = BeautifulSoup(r.content, "lxml")

    try:
        title = soup.find('div', {'class': 'stitle'}).text
    except Exception as e:
        title = None

    try:
        body = soup.find('div', {'class': 'scontent portfolio__content'}).get_text(separator="\n")
    except Exception as e:
        body = None

    data = {
        'url': url,
        'title': title,
        'body': body,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

5637it [1:46:32,  1.13s/it]


In [14]:
from glob import glob

files = glob('page/*.json')
len(files)

5637

In [15]:
with open('ringgitplus.com.jsonl', 'w') as fopen_l:
    for f in files:
        with open(f) as fopen:
            d = json.load(fopen)
        fopen_l.write(f'{json.dumps(d)}\n')